In [ ]:
import time
import numpy as np
import pandas as pd
import os
from os.path import join, abspath, dirname, realpath
import sys
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta
from dateutil import relativedelta
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, roc_auc_score


from dateutil.relativedelta import relativedelta

from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc
import argparse
import csv


import multiprocessing as mp
from multiprocessing import Process

from functools import partial
from skopt import gp_minimize
from skopt import space

from functions import *
# testarw kati. bgalto apo comment an den treksei olo 
#from Train_bayes import *
from Train_bayes_pr import *
import shap


In [ ]:
#preparation of the data
def data_prep(data):
    
    data['date']=pd.to_datetime(data['date'])
    data['migraine_start']=data['migraine_start'].astype(int)
    data['order']=data['order'].astype(float).astype(int)
    data['imputation']=data['imputation'].astype(float).astype(int)
    data.loc[:, ['temp_avg','sun_perc','precip_tot',
                  'pres_avg','cloud_avg','wind_avg','hum_avg',]] = data.loc[:, ['temp_avg','sun_perc','precip_tot','pres_avg',
                  'cloud_avg','wind_avg','hum_avg',]].astype(float)
    
    data=data.sort_values(['order', 'date'])
    data.reset_index(drop=True, inplace=True)
    return(data)

In [ ]:
#files for lstm: performance_lstm_train.zip and performance_lstm_test.zip

#files for gru: performance_gru_train.zip and performance_gru_test.zip

#reads predictions from every model
def get_predictions(path_for_train,path_for_test):
    train_values_p = pd.read_csv(path_for_train, sep=',', index_col=False, dtype='unicode')
    test_values_p = pd.read_csv(path_for_test, sep=',', index_col=False, dtype='unicode')

    train_values_p['1']=train_values_p['1'].astype(float).astype(int)
    train_values_p['0']=train_values_p['0'].astype(float)
    test_values_p['1']=test_values_p['1'].astype(float).astype(int)
    test_values_p['0']=test_values_p['0'].astype(float)
    (train_values_p).columns=['pred','real']
    (test_values_p).columns=['pred','real']

    return(train_values_p,test_values_p)

#evaluates AUC pr of the predictions
def evalute_results(train_values_p,test_values_p):
    precision, recall, thresholds = precision_recall_curve(train_values_p['real'].to_numpy(), 
                                                           train_values_p['pred'].to_numpy())
    auc_pr_train=auc(recall, precision)

    precision, recall, thresholds = precision_recall_curve(test_values_p['real'].to_numpy(), 
                                                           test_values_p['pred'].to_numpy())
    auc_pr_test=auc(recall, precision)
    print(f'AUC PR: Test={round(auc_pr_test,3)}, Train={round(auc_pr_train,3)}')

In [ ]:
#generates AUC_pr for every patient individually
def results_individually(test_values_p):
    results=pd.DataFrame(np.unique(real_test.patient_id), columns=['patient_id'])
    auc_pr=[]
    baseline_pr=[]
    days=[]
    #data of every patient are processed individually
    for patient in np.unique(real_test.patient_id):
        #data of every patient
        data_p=real_test.loc[real_test.patient_id==patient]
        precision, recall, thresholds = precision_recall_curve(test_values_p['real'][data_p.index].to_numpy(), 
                                                               test_values_p['pred'][data_p.index].to_numpy())
        auc_pr.append(auc(recall, precision))
        #baseline of every patient calculated with migraines and total number of entries
        baseline_pr.append(data_p.loc[data_p['migraine_start']==1].shape[0]/data_p.shape[0])
        days.append(data_p.shape[0])
    results['days']=days
    results['auc_pr']=auc_pr
    results['baseline']=baseline_pr
    results=results.loc[(results['baseline']!=0)]

    return results


In [ ]:
#baselines calculation for train and test set
train = pd.read_csv('train_plus.csv', sep=',', index_col=False, dtype='unicode')#, error_bad_lines=False
train = data_prep(train)

test = pd.read_csv('test_plus.csv', sep=',', index_col=False, dtype='unicode')#, error_bad_lines=False
test = data_prep(test)

#BASELINE= (migraine onsets) / (total number of entries)
real_train=train.loc[(train['imputation']==0)]
pr_train=real_train.loc[real_train['migraine_start']==1].shape[0]/real_train.shape[0]
real_test=test.loc[(test['imputation']==0)]
pr_test=real_test.loc[real_test['migraine_start']==1].shape[0]/real_test.shape[0]
real_test=real_test.reset_index()

print(f'Baseline Train AUC PR: {round(pr_train,3)}')
print(f'Baseline Test AUC PR: {round(pr_test,3)}')

# LSTM

In [ ]:
#Files with the results of LSTM: 'performance_lstm_train.zip' & 'performance_lstm_test.zip'
train_values_p,test_values_p=get_predictions('performance_lstm_train.zip','performance_lstm_test.zip')
evalute_results(train_values_p,test_values_p)

results_lstm=results_individually(test_values_p)

In [ ]:
#ratio of patients with AUC Pr better their baseline
sum((results_lstm.auc_pr-results_lstm.baseline)>0)/results_lstm.shape[0]

In [ ]:
#exclude patients with baseline==0
results_lstm=results_lstm.loc[(results_lstm['baseline']!=0)]
#scatterplot with aucpr- Baseline for every patient
sns.scatterplot(x=results_lstm['days'],
                y=(results_lstm['auc_pr'])-results_lstm['baseline'],
                hue=results_lstm['baseline'])
plt.xlabel('Total number of days')
plt.axhline(y=0.1390,color='blue',linestyle='--')

plt.axhline(y=0,color='red')
plt.ylabel('AUC PR - Baseline')

In [ ]:
a=results_lstm['auc_pr']-results_lstm['baseline']
#top 80 percent for the best AUC pr - Baseline
p = np.percentile(a, 80) 
#good_results_lstm=results_lstm.loc[(a>p)&(results_lstm['baseline']!=0)].patient_id
p

In [ ]:

results_lstm=results_lstm.loc[(results_lstm['baseline']!=0)]
results_lstm['migraines']=(results_lstm['days']*results_lstm['baseline']).astype(int)

sns.scatterplot(x=results_lstm['migraines'],
                y=(results_lstm['auc_pr'])-results_lstm['baseline'],
                hue=results_lstm['baseline'])
plt.xlabel('Migraine days')
plt.axhline(y=0.1390,color='blue',linestyle='--')

plt.axhline(y=0,color='red')
plt.ylabel('AUC PR - Baseline')

# GRU

In [ ]:
train_values_p,test_values_p=get_predictions('performance_gru_train.zip','performance_gru_test.zip')
evalute_results(train_values_p,test_values_p)
results_gru=results_individually(test_values_p)

In [ ]:
results_gru=results_gru.loc[(results_gru['baseline']!=0)]
sns.scatterplot(x=results_gru['days'],
                y=(results_gru['auc_pr'])-results_gru['baseline'],
                hue=results_gru['baseline'])
plt.xlabel('Days')

plt.axhline(y=0,color='red')
plt.title("GRU")
plt.ylabel('AUC PR - Baseline')

## LSTM

In [ ]:
a=results_lstm['auc_pr']-results_lstm['baseline']
p = np.percentile(a, 80) 
good_results_lstm=results_lstm.loc[(a>p)&(results_lstm['baseline']!=0)].patient_id


## GRU

In [ ]:
a=results_gru['auc_pr']-results_gru['baseline']
p = np.percentile(a, 80) 
good_results_gru=results_gru.loc[(a>p)&(results_gru['baseline']!=0)].patient_id

## Comparison LSTM-GRU

In [ ]:
same_patients=(set(good_results_lstm)&set(good_results_gru))

results_gru=results_gru.loc[(results_gru['baseline']!=0)]
sns.scatterplot(x=results_lstm.loc[results_lstm['patient_id'].isin(same_patients)]['auc_pr'],
                y=results_gru.loc[results_gru['patient_id'].isin(same_patients)]['auc_pr'],
                hue=results_gru['days'])
x=np.linspace(0,1,100)
y=x
plt.text(0.86,0.905,'y=x',color='red',rotation=45,fontsize=12)
plt.plot(x,y,'-r')
plt.xlabel('AUC PR for LSTM')
plt.ylabel('AUC PR for GRU')



# Exaplainability with LSTM

## Each patient seperatly

In [ ]:
train_values_p,test_values_p=get_predictions('performance_lstm_train.zip','performance_lstm_test.zip')

#function to get the results for the performance
def fakefunction(x):
    return(test_values_p['pred'][x.index])

#the shap plot for every patient is produced and displayed individually
for patient in good_results_lstm:
    data_p=real_test.loc[real_test.patient_id==patient]
    
    explainer = shap.Explainer(fakefunction, data_p.iloc[:,4:-2])
    shap_values = explainer(data_p.iloc[:,4:-2])
    shap.plots.bar(shap_values)


## All patients together

In [ ]:
data_p=real_test

explainer = shap.Explainer(fakefunction, data_p.iloc[:,4:-2])
shap_values = explainer(data_p.iloc[:,4:-2])

shap.plots.bar(shap_values)


In [ ]:
#shap.plots.beeswarm(shap_values,ylabel=1)
fig=shap.summary_plot(shap_values,show=False,
                      feature_names=['Temperature','Sun','Precipitation','Pressure','Cloud','Wind','Humidity'])
plt.xlim([-0.0027,0.00095])
plt.xticks(fontsize=9)

plt.show()